In [ ]:
%matplotlib notebook
#from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
#from matplotlib import cm

from functools import partial

import numpy as np
from scipy.optimize import minimize

# setting up the reference discretization

In [ ]:
%run ../scripts/OS2015_academic_problem.py

config = {'num_coarse_grid_elements': [4, 4], 
          'num_grid_refinements': 2,
          'num_grid_subdomains': [2, 2], 
          'num_grid_oversampling_layers': 4, # num_grid_oversampling_layers has to exactly cover one subdomain!
          'initial_RB_order': 0,
          'enrichment_target_error': -1,
          'marking_doerfler_theta': -1,
          'marking_max_age': -1}

grid_and_problem_data = init_grid_and_problem(config)

mu_bar = grid_and_problem_data['mu_bar']
parameter_range = grid_and_problem_data['parameter_range']
parameter_range = (parameter_range[0], parameter_range[1])

initial_guess = 0.5*(parameter_range[0] + parameter_range[1])

In [ ]:
%run ../scripts/discretize_elliptic.py

block_SWIPDG_disc, block_space, enrichment_local_boundary_info = discretize(grid_and_problem_data)
block_SWIPDG_disc.disable_logging()

parameter_space = block_SWIPDG_disc.parameter_space
parameter_type = block_SWIPDG_disc.parameter_type
parse_parameter = block_SWIPDG_disc.parse_parameter
mu_bar = parse_parameter(mu_bar)

In [ ]:
# this is required to estimate the target error for the online enrichment
print('estimating some detailed errors:')
detailed_errors = []
for mu in parameter_range:
    mu = parse_parameter(mu)
    print('  {}: '.format(mu), end='', flush=True)
    U = block_SWIPDG_disc.solve(mu)
    estimate = block_SWIPDG_disc.estimate(U, mu=mu)
    print(estimate)
    detailed_errors.append(estimate)

In [ ]:
# as a result of the above, we set
config['enrichment_target_error'] = 1.
config['marking_doerfler_theta'] = 0.33
config['marking_max_age'] = 2

# minimizing using the reference discretization

In [ ]:
def reference_quantity_of_interest(mu):
    return block_SWIPDG_disc.rhs.apply(block_SWIPDG_disc.solve(mu)).data[0]

print('computing some detailed quantities of interest ... ', end='')

training_set = parameter_space.sample_uniformly(5)
training_set.extend(parameter_space.sample_randomly(10))
training_set = [mu['diffusion'] for mu in training_set]
training_set.sort()
reference_quantities_of_interest = [reference_quantity_of_interest(mu) for mu in training_set]

print('done')

plt.plot(training_set, reference_quantities_of_interest, '-o', label='QoI')
plt.title('QoI over training set')
plt.xlabel('parameter range')
plt.ylabel('QoI')
plt.legend()

In [ ]:
print('minimizing reference quantity of interest ', end='')

reference_minimization_data = {'num_evals': 0,
                               'all_evaluation_points': [],
                               'evaluation_points': [initial_guess, ]}


def quantity_of_interest(function, cfg, mu):
    print('.', end='')
    cfg['num_evals'] += 1
    cfg['all_evaluation_points'].append(mu)
    return function(mu)


def report(result, evaluate_qoi, cfg):
    if (result.status != 0):
        print(' failed!')
    else:
        print(' succeded!')
        print('  mu_min:    {}'.format(parse_parameter(result.x)))
        print('  QoI(mu_min): {}'.format(result.fun[0]))
        print('  num iterations:     {}'.format(result.nit))
        print('  num function calls: {}'.format(cfg['num_evals']))

result = minimize(partial(quantity_of_interest, reference_quantity_of_interest, reference_minimization_data),
                  initial_guess,
                  method='L-BFGS-B', jac=False,
                  bounds=(parameter_range,),
                  callback=lambda xk: reference_minimization_data['evaluation_points'].append(xk),
                  options={'ftol': 1e-15, 'gtol': 1e-15})

report(result, reference_quantity_of_interest, reference_minimization_data)

plt.plot(training_set, reference_quantities_of_interest, label='QoI')
plt.plot(reference_minimization_data['all_evaluation_points'],
         [reference_quantity_of_interest(mu) for mu in reference_minimization_data['all_evaluation_points']],
         'o', label='minimization points')
plt.title('parameter values selected during optimization')
plt.xlabel('paramter range')
plt.ylabel('QoI')
plt.legend()

# setting up the standard RB discretization

In [ ]:
print('reducing with standard RB:', flush=True)

from pymor.algorithms.greedy import greedy
from pymor.discretizations.basic import StationaryDiscretization
from pymor.parameters.functionals import GenericParameterFunctional
from pymor.reductors.coercive import CoerciveRBReductor

def coercivity_estimator(mu):
    return 1./np.sqrt(alpha(grid_and_problem_data['lambda']['coefficients'], mu, mu_bar))

SWIPDG_disc = StationaryDiscretization(
    block_SWIPDG_disc.operators['global_op'],
    block_SWIPDG_disc.operators['global_rhs'],
    products={'energy_dg_mu_bar': block_SWIPDG_disc.operators['global_op'].assemble(mu_bar)},
    parameter_space=parameter_space,
    name='SWIPDG')

RB_reductor = CoerciveRBReductor(
    SWIPDG_disc,
    product=SWIPDG_disc.energy_dg_mu_bar_product,
    coercivity_estimator=GenericParameterFunctional(coercivity_estimator, parameter_type))

RB_greedy_data = greedy(SWIPDG_disc, RB_reductor, training_set,
                        extension_params={'method': 'gram_schmidt'},
                        max_extensions=len(training_set))

num_RB_greedy_extensions = RB_greedy_data['extensions']
RB_greedy_mus, RB_greedy_errors = RB_greedy_data['max_err_mus'], RB_greedy_data['max_errs']
RB_greedy_mus = [mu[0] for mu in RB_greedy_mus]

print('RB system is of siez {}x{}'.format(num_RB_greedy_extensions, num_RB_greedy_extensions))
print('maximum estimated model reduction error over training set: {}'.format(RB_greedy_errors[-1]))

In [ ]:
RB_rd = RB_greedy_data['reduced_discretization']
RB_rd.disable_logging()

plt.plot(list(range(num_RB_greedy_extensions)) + [len(range(num_RB_greedy_extensions))],
         RB_greedy_errors)
plt.yscale('log')
plt.title('error decay during greedy basis extension')
plt.xlabel('greedy extension step')
plt.ylabel('max estimted relative error over training set')

# minimizing using the standard RB discretization

In [ ]:
def RB_quantity_of_interest(mu):
    return RB_rd.rhs.apply(RB_rd.solve(mu)).data[0]

plt.plot(training_set, reference_quantities_of_interest, label='reference QoI')
plt.scatter(RB_greedy_mus,
            [RB_quantity_of_interest(mu) for mu in RB_greedy_mus],
            s=np.exp(6.*np.linspace(1, 0.1, len(RB_greedy_mus))),
            label='selected point')
plt.title('parameter selection during greedy basis generation (larger: picked earlier)')
plt.xlabel('paramter range')
plt.ylabel('QoI')
plt.legend()

In [ ]:
print('computing some RB quantities of interest ... ', end='')
RB_quantities_of_interest = [RB_quantity_of_interest(mu) for mu in training_set]
print('done')

print('L-infty error of QoI over training set: {}'.format(
    np.max(np.abs(np.array(reference_quantities_of_interest) - np.array(RB_quantities_of_interest)))
))

plt.plot(training_set, reference_quantities_of_interest, label='reference QoI')
plt.plot(training_set, RB_quantities_of_interest, 'o', label='RB QoI')
plt.title('comparison of reference and reduced QoI over training set')
plt.xlabel('parameter range')
plt.ylabel('QoI')
plt.legend()

In [ ]:
print('minimizing reduced quantity of interest ', end='')

RB_minimization_data = {'num_evals': 0,
                        'all_evaluation_points': [],
                        'evaluation_points': [initial_guess, ]}

result = minimize(partial(quantity_of_interest, RB_quantity_of_interest, RB_minimization_data),
                  initial_guess,
                  method='L-BFGS-B', jac=False,
                  bounds=(parameter_range,),
                  callback=lambda xk: RB_minimization_data['evaluation_points'].append(xk),
                  options={'ftol': 1e-15, 'gtol': 1e-15})

report(result, RB_quantity_of_interest, RB_minimization_data)

plt.plot(training_set, reference_quantities_of_interest, label='reference QoI')
plt.plot(reference_minimization_data['all_evaluation_points'],
         [reference_quantity_of_interest(mu) for mu in reference_minimization_data['all_evaluation_points']],
         'o', label='reference minimization points')
plt.plot(RB_minimization_data['all_evaluation_points'],
         [RB_quantity_of_interest(mu) for mu in RB_minimization_data['all_evaluation_points']],
         'kx', label='RB minimization points')
plt.title('parameter values selected during optimization')
plt.xlabel('paramter range')
plt.ylabel('QoI')
plt.legend()

# setting up the LRBMS discretization

In [ ]:
%run ../scripts/offline.py

LRBMS_reductor = init_local_reduced_bases(block_SWIPDG_disc, block_space, config['initial_RB_order'])

#from pymor.core.exceptions import ExtensionError
#
#print('adding some global solution snapshots to reduced basis ...', flush=True)
#for mu in parameter_range:
#    U = d.solve(mu)
#    try:
#        reductor_blocked.extend_basis(U)
#    except ExtensionError:
#        pass
#print('')

print('reducing:', flush=True)
LRBMS_rd = LRBMS_reductor.reduce()
LRBMS_rd = LRBMS_rd.with_(estimator=block_SWIPDG_disc.estimator)
LRBMS_rd.disable_logging()
print('initial reduced (LRBMS) system is of size {}x{}'.format(LRBMS_rd.solution_space.dim, LRBMS_rd.solution_space.dim))

# minimizing using the LRBMS discretization

In [ ]:
def LRBMS_quantity_of_interest(mu):
    return LRBMS_rd.rhs.apply(LRBMS_rd.solve(mu)).data[0]

print('computing some initial LRBMS reduced quantities of interest ...')

LRBMS_quantities_of_interest = [LRBMS_quantity_of_interest(mu) for mu in training_set]

print('L-infty error w.r.t. reference QoI: {}'.format(
    np.max(np.abs(np.array(reference_quantities_of_interest) - np.array(LRBMS_quantities_of_interest)))
))
print('L-infty error w.r.t RB QoI: {}'.format(
    np.max(np.abs(np.array(RB_quantities_of_interest) - np.array(LRBMS_quantities_of_interest)))
))

plt.plot(training_set, reference_quantities_of_interest, label='reference QoI')
plt.plot(training_set, RB_quantities_of_interest, 'o', label='RB QoI')
plt.plot(training_set, LRBMS_quantities_of_interest, 'x', label='initial LRBMS QoI')
plt.title('comparison of reference, RB and initial LRBMS QoI over training set')
plt.xlabel('parameter range')
plt.ylabel('QoI')
plt.legend()

In [ ]:
print('minimizing initial LRBMS quantity of interest ', end='')

LRBMS_minimization_data = {'num_evals': 0,
                           'all_evaluation_points': [],
                           'evaluation_points': [initial_guess, ]}

result = minimize(partial(quantity_of_interest, LRBMS_quantity_of_interest, LRBMS_minimization_data),
                  initial_guess,
                  method='L-BFGS-B', jac=False,
                  bounds=(parameter_range,),
                  callback=lambda xk: LRBMS_minimization_data['evaluation_points'].append(xk),
                  options={'ftol': 1e-15, 'gtol': 1e-15})

report(result, LRBMS_quantities_of_interest, LRBMS_minimization_data)

plt.plot(training_set, reference_quantities_of_interest, label='reference QoI')
plt.plot(reference_minimization_data['all_evaluation_points'],
         [reference_quantity_of_interest(mu) for mu in reference_minimization_data['all_evaluation_points']],
         'o', label='reference minimization points')
plt.plot(RB_minimization_data['all_evaluation_points'],
         [RB_quantity_of_interest(mu) for mu in RB_minimization_data['all_evaluation_points']],
         'kx', label='RB minimization points')
plt.plot(LRBMS_minimization_data['all_evaluation_points'],
         [LRBMS_quantity_of_interest(mu) for mu in LRBMS_minimization_data['all_evaluation_points']],
         'o', label='LRBMS minimization points')
plt.title('parameter values selected during optimization')
plt.xlabel('paramter range')
plt.ylabel('QoI')
plt.legend()